In [6]:
import os
import pandas

mirna_src_dir = os.getcwd() + "/assn-mirna-luad/data/processed/miRNA/"
clinical_src_dir = os.getcwd() + "/assn-mirna-luad/data/processed/clinical/"

mirna_tumor_df = pandas.read_csv(mirna_src_dir+'tumor_miRNA.csv')
mirna_normal_df = pandas.read_csv(mirna_src_dir+'normal_miRNA.csv')
clinical_df = pandas.read_csv(clinical_src_dir+'clinical.csv')

print "mirna_tumor_df.shape", mirna_tumor_df.shape
print "mirna_normal_df.shape", mirna_normal_df.shape

"""
Here we select samples to use for our regression analysis
"""
matched_samples = pandas.merge(clinical_df, mirna_tumor_df, on='patient_barcode')['patient_barcode']
print "matched_samples", matched_samples.shape
# merged = pandas.merge(clinical_df, mirna_normal_df, on='patient_barcode')
# print merged.shape
# print
# print merged['histological_type'].value_counts().sort_index(axis=0)
# print
# print merged['pathologic_stage'].value_counts().sort_index(axis=0)
# print
# print merged['pathologic_T'].value_counts().sort_index(axis=0)
# print
# print merged['pathologic_N'].value_counts().sort_index(axis=0)
# print
# print merged['pathologic_M'].value_counts().sort_index(axis=0)
# print

mirna_tumor_df.shape (513, 1882)
mirna_normal_df.shape (46, 1882)
matched_samples (513,)


## We now create data matrixes from tumor vs normal samples

In [ ]:
from sklearn import preprocessing

X = mirna_df[mirna_df['patient_barcode'].isin(matched_samples)]
X.__delitem__('patient_barcode')
Y = clinical_df[clinical_df['patient_barcode'].isin(matched_samples)]['pathologic_stage']
print 'miRNA null values', X.isnull().sum().sum()
print 'pathologic null values', Y.isnull().sum()
print 'X.shape', X.shape
print 'Y.shape', Y.shape
mirna_list = X.columns.values

X_scaler = preprocessing.StandardScaler(with_mean=False).fit(X)
X = X_scaler.transform(X)
# Y_scaler = preprocessing.StandardScaler().fit(Y)
# Y = Y_scaler.transform(Y)

## Perform classification with LASSO

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn import linear_model

X_train, X_test, y_train, y_test = train_test_split(X, Y.as_matrix(), test_size=0.1, random_state=np.random.randint(0, 10000))
print 'X_train', X_train.shape, ', y_train', y_train.shape
print 'X_test', X_test.shape, ', y_test', y_test.shape

lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)